In [ ]:
#As always, we import everything
import pandas as pd
import os
import re
import hdf5_getters as getters
import requests
from bs4 import BeautifulSoup

In [ ]:
all_files = []
for (dirpath, dirnames, filenames) in os.walk("MillionSongSubset/data"):
    all_files.extend([dirpath + "/" + filename for filename in filenames if filename.endswith(".h5")])

In [ ]:
def get_track_id(filename):
    h5 = getters.open_h5_file_read(filename)
    track_id = getters.get_track_id(h5)
    h5.close()
    return track_id

In [ ]:
def get_title(filename):
    h5 = getters.open_h5_file_read(filename)
    title = getters.get_title(h5).decode()
    h5.close()
    return title

In [ ]:
def get_artist_name(filename):
    h5 = getters.open_h5_file_read(filename)
    artist_name = getters.get_artist_name(h5).decode()
    h5.close()
    return artist_name

In [ ]:
"""df = pd.read_csv('MillionSongSubset/msd_genre_dataset.txt')
genre_dataset = df[['genre', 'track_id']].set_index('track_id')
genre_dataset.to_csv('MillionSongSubset/genre_dataset.txt')"""
genre_dataset = pd.read_csv('MillionSongSubset/genre_dataset.txt').set_index('track_id')

In [ ]:
def get_song_genres(track_id):
    if track_id in genre_dataset.index:
        return "&".join(genre_dataset.loc[[track_id]].values[0][0].split(' and '))
    else:
        return None

In [ ]:
"""data = pd.DataFrame([])

for filename in all_files:
    track_id = get_track_id(filename).decode()
    genres = get_song_genres(track_id)
    if genres:
        data = data.append(pd.DataFrame({'track_id': track_id, 'genres': genres, 'artist_name' : get_artist_name(filename), 'title' : get_title(filename), 'lyrics' : ""}, index=[0]))

data.set_index('track_id', inplace=True)
data.to_csv('data/data.csv')
"""
data = pd.read_csv('data/data.csv').set_index('track_id')

In [ ]:
lyrics_df1 = pd.read_csv('lyrics/songdata1.csv')
lyrics_df1.set_index(['artist', 'song'], inplace=True)

In [ ]:
lyrics_df2_raw = pd.read_csv('lyrics/songdata2.csv')
lyrics_df2 = lyrics_df2_raw[['song', 'artist', 'lyrics']].set_index(['artist', 'song'])

In [ ]:
def get_lyrics_csv1(artist_name, title):
    if (artist_name, title) in lyrics_df1.index:
        return lyrics_df1.loc[artist_name, title].values[0][1]
    else:
        return None

In [ ]:
def get_lyrics_csv2(artist_name, title):
    index_artist_name = artist_name.lower().replace(' ', '-')
    index_title = title.lower().replace(' ', '-')
    if (index_artist_name, index_title) in lyrics_df2.index:
        lyrics = lyrics_df2.loc[index_artist_name, index_title].values[0][0]
        if len(lyrics) == 0:
            return None
        else:
            print("found with csv2")
            return lyrics
    else:
        return None

In [ ]:
def get_lyrics_genius(artist_name, title):
    url = (artist_name.lower().replace(' ', '-') + '-' + re.sub(r'\([^)]*\)', '', title).rstrip().lower().replace(' ', '-') + '-lyrics').capitalize().replace("'", '')
    r = requests.get("https://genius.com/" + url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    if len(soup.select(".render_404-headline")) > 0:
        return None
    
    return soup.select(".lyrics")[0].text

In [ ]:
def get_lyrics(artist_name, title):
    lyrics = get_lyrics_csv1(artist_name, title)
    if lyrics:
        return lyrics
    
    lyrics = get_lyrics_csv2(artist_name, title)
    if lyrics:
        return lyrics
    
    lyrics = get_lyrics_genius(artist_name, title)
    if lyrics:
        return lyrics
    
    return ""

In [ ]:
#Match lyrics
for index, row in data.iterrows():
    lyrics = get_lyrics(row['artist_name'], row['title'])
    if len(lyrics) > 0:
        print(row['artist_name'] + " " + row['title'] + " not found")
    else:
        print(row['artist_name'] + " " + row['title'] + " not found")
    data.loc[index, 'lyrics'] = lyrics
    
data